In [135]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

d = ctrl.Antecedent(np.arange(0, 101, 1), 'zadany_zasieg')
alpha = ctrl.Antecedent(np.arange(0, 71, 1), 'kat_rzutu')
k = ctrl.Antecedent(np.arange(0, 1.2, 0.1), 'wspolczynnik_oporu')
v0 = ctrl.Consequent(np.arange(0, 101, 1), 'predkosc_poczatkowa')

d['s'] = fuzz.trimf(d.universe, [0, 0, 35])
d['m'] = fuzz.trimf(d.universe, [30, 50, 75])
d['l'] = fuzz.trimf(d.universe, [70, 100, 100])

alpha['s'] = fuzz.trimf(alpha.universe, [0, 0, 30])
alpha['m'] = fuzz.trimf(alpha.universe, [20, 35, 50])
alpha['l'] = fuzz.trimf(alpha.universe, [40, 70, 70])

k['s'] = fuzz.trimf(k.universe, [0, 0, 0.35])
k['m'] = fuzz.trimf(k.universe, [0.3, 0.5, 0.8])
k['l'] = fuzz.trimf(k.universe, [0.7, 1.2, 1.2])

v0['s'] = fuzz.trimf(v0.universe, [0, 0, 50])
v0['m'] = fuzz.trimf(v0.universe, [25, 50, 75])
v0['l'] = fuzz.trimf(v0.universe, [50, 100, 100])

# d - zasieg rzutu, alpha - kat rzutu, k - wsp oporu powietrza
rules = [
    ctrl.Rule(d['s'] & alpha['s'] & k['s'], v0['s']),
    ctrl.Rule(d['s'] & alpha['m'] & k['s'], v0['s']),
    ctrl.Rule(d['s'] & alpha['l'] & k['s'], v0['s']),

    ctrl.Rule(d['s'] & alpha['s'] & k['m'], v0['m']),
    ctrl.Rule(d['s'] & alpha['m'] & k['m'], v0['m']),
    ctrl.Rule(d['s'] & alpha['l'] & k['m'], v0['l']),

    ctrl.Rule(d['s'] & alpha['s'] & k['l'], v0['m']),
    ctrl.Rule(d['s'] & alpha['m'] & k['l'], v0['l']),
    ctrl.Rule(d['s'] & alpha['l'] & k['l'], v0['l']),


    ctrl.Rule(d['m'] & alpha['s'] & k['s'], v0['s']),
    ctrl.Rule(d['m'] & alpha['m'] & k['s'], v0['s']),
    ctrl.Rule(d['m'] & alpha['l'] & k['s'], v0['m']),

    ctrl.Rule(d['m'] & alpha['s'] & k['m'], v0['m']),
    ctrl.Rule(d['m'] & alpha['m'] & k['m'], v0['l']),
    ctrl.Rule(d['m'] & alpha['l'] & k['m'], v0['l']),

    ctrl.Rule(d['m'] & alpha['s'] & k['l'], v0['l']),
    ctrl.Rule(d['m'] & alpha['m'] & k['l'], v0['l']),
    ctrl.Rule(d['m'] & alpha['l'] & k['l'], v0['l']),

    ctrl.Rule(d['l'] & alpha['s'] & k['s'], v0['s']),
    ctrl.Rule(d['l'] & alpha['m'] & k['s'], v0['m']),
    ctrl.Rule(d['l'] & alpha['l'] & k['s'], v0['m']),

    ctrl.Rule(d['l'] & alpha['s'] & k['m'], v0['l']),
    ctrl.Rule(d['l'] & alpha['m'] & k['m'], v0['l']),
    ctrl.Rule(d['l'] & alpha['l'] & k['m'], v0['l']),

    ctrl.Rule(d['l'] & alpha['s'] & k['l'], v0['l']),
    ctrl.Rule(d['l'] & alpha['m'] & k['l'], v0['l']),
    ctrl.Rule(d['l'] & alpha['l'] & k['l'], v0['l'])
]

model = ctrl.ControlSystem(rules)
v0est = ctrl.ControlSystemSimulation(model)

def predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc):
    v0est.input['zadany_zasieg'] = zadany_zasieg_wartosc
    v0est.input['kat_rzutu'] = kat_rzutu_wartosc
    v0est.input['wspolczynnik_oporu'] = wspolczynnik_oporu_wartosc
    v0est.compute()
    return v0est.output['predkosc_poczatkowa']

zadany_zasieg_wartosc = 10
kat_rzutu_wartosc = 10
wspolczynnik_oporu_wartosc = 0.1

predkosc_pocz = predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc)
print("Szacowana prędkość początkowa dla zasięgu rzutu", zadany_zasieg_wartosc, "to:", predkosc_pocz)


Szacowana prędkość początkowa dla zasięgu rzutu 10 to: 18.055555555555554


In [134]:
import math

def zasieg_rzutu(V0, alpha, k):
    g = 9.81
    xmax = (V0**2 * math.sin(math.radians(2 * alpha))) / (g + k * V0)
    return xmax

# Metoda zlotego podzialu
def znajdz_poczatkowa_predkosc(d, alfa, k, epsilon=0.1):
    a = 0.1
    b = 100
    
    while True:
        V0 = (a + b) / 2
        xmax = zasieg_rzutu(V0, alfa, k)

        if abs(xmax - d) < epsilon:
            return V0
        elif xmax < d:
            a = V0
        else:
            b = V0

zasieg = 10
kat = 10
opor_powietrza = 0.1

V0 = znajdz_poczatkowa_predkosc(zasieg, kat, opor_powietrza)
print("Prędkość początkowa rzutu ukośnego:", V0, "m/s")


Prędkość początkowa rzutu ukośnego: 18.441015625 m/s
